# I/F Calculator Correction Sheet

In [2]:
import matplotlib.pyplot as plt
import astropy.io.fits as ast
import numpy as np
import csv

from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename 

from photutils import centroid_com

print("Required Libraries Loaded Successfully...")

%matplotlib inline
plt.ion()
print("Matplotlib Intialised...")

import tools
print("Imported Oliver King's Planetary Science Tools Successfully...")

def AU_to_km(n):
    Dist = n*149598000
    return Dist
print("Defined all Necessary Unit Conversion Routines...")

print("\nAll Modules Imported Successfully!")

#initialises the crucial variables
filepath = "OPAL RAWS/Cycle_26/1/"
filters = ['F467M','F547M','FQ619N','F657N','FQ727N','F763M','F845M']
wavelength = ['467','547','619','657','727','763','845']
SolFlux = [2019.04,1859.82,1707.85,1238,1339.23,1541.91,1023.5] #Solar Flux in each filter wavelengths according to Amy Simon's Spreadsheet 
IF_Factors_OPAL = [0.00475,0.00249,0.00160,0.00405,0.00203,0.00145,0.00130] #taken from the OPAL ReadMe
fitsdata = ['','','','','','','']
resolution = ['','','','','','','']
imgdata = ['','','','','','','']
fitshdr = ['','','','','','','']
scihdr = ['','','','','','','']
ephemerides = ['','','','','','','']
CLEANUPFLAG = False

for i in range(len(filters)):
    file = get_pkg_data_filename(filepath+filters[i]+'/'+wavelength[i]+'_1_drz.fits')
    fitsdata[i] = fits.open(file)
    imgdata[i] = fits.getdata(file, ext=1)
    resolution[i] = imgdata[i].shape
    fitshdr[i] = fitsdata[i][0].header
    scihdr[i] = fitsdata[i][1].header
    #899 = Neptune Moving Target
    #@hst = Hubble Space Telescope
    #reads epoch from header[exposure start time]
    ephemerides[i] = tools.mapping.get_ephemerides(target = 'neptune', loc = 'hst', epoch = fitshdr[i]['EXPSTART']) 

Required Libraries Loaded Successfully...
Matplotlib Intialised...
Imported Oliver King's Planetary Science Tools Successfully...
Defined all Necessary Unit Conversion Routines...

All Modules Imported Successfully!


In [13]:
#Defines the constants needed to calculate I/F conversion factor that are based on the WFC3/UVIS Instrument
#PixWidth = plate scale of WFC3/UVIS, taken from Amy Simon's Uranus Spreadsheet, and checked with the WFC3 Manual
PixWidth = 0.04 #arcseconds 
#omega = Solid Angle of a Single WFC3/UVIS Pixel
omega = ((PixWidth/3600)*(np.pi/180))**2 #steradians

#initialises the array of I/F factors for the imported imageset
IF_Factors = np.zeros(len(filters))
Distances = np.zeros(len(filters))
OPAL_dist = np.zeros(len(filters))
eph_dist = np.zeros(len(filters))
calc_PF = np.zeros(len(filters))
amy_PF = [1.639547e-18,4.5253925e-19,4.677e-18,2.14434e-18,4.653e-18,3.8165376e-19,4.5375198e-19]
hdr_PF = np.zeros(len(filters))


#determines I/F factors with the above formula for each image in the set
for i in range(len(filters)):
    fractop = ((scihdr[i]['PHOTFLAM']*10)/omega)
    fracbottom = ((1.0/ephemerides[i]['r'])**2)*(SolFlux[i]/np.pi)
    IF_Factors[i] = fractop/fracbottom
    IF_Factors[i] = round(IF_Factors[i],5)
    
    Distances[i] = (((scihdr[i]['PHOTFLAM']*10)/omega)/(IF_Factors[i]*(SolFlux[i]/np.pi)))**(-0.5)
    OPAL_dist[i] = (((scihdr[i]['PHOTFLAM']*10)/omega)/(IF_Factors_OPAL[i]*(SolFlux[i]/np.pi)))**(-0.5)
    eph_dist[i] = ephemerides[i]['r']
    
    hdr_PF[i] = scihdr[i]['PHOTFLAM']
    calc_PF[i] = (omega*(IF_Factors[i]*((1.0/ephemerides[i]['r'])**2)*(SolFlux[i]/np.pi)))/10
    
print('I/F Factors I Calculated: '+str(IF_Factors))    
print('I/F Factors OPAL Readme : '+str(IF_Factors_OPAL))
print('')
print('My Distance Calculated  : '+str(Distances))
print('OPAL Distance Calculated: '+str(OPAL_dist))
print('Distance Ephemerides    : '+str(eph_dist))
print('')
print('My Determined PHOTFLAM: '+str(calc_PF))
print('Header PHOTFLAM       : '+str(hdr_PF))
print('Spreadsheet PHOTFLAM  : '+str(amy_PF))

I/F Factors Calculated : [0.00063 0.00019 0.00207 0.00132 0.00265 0.00019 0.00034]
I/F Factors OPAL Readme: [0.00475, 0.00249, 0.0016, 0.00405, 0.00203, 0.00145, 0.0013]

My Distance Calculated  : [29.98781315 30.14968326 29.9473752  29.89762994 29.94832236 30.06621391
 30.14694097]
OPAL Distance Calculated: [ 82.34198387 109.1454478   26.32895258  52.36936502  26.21184439
  83.0588557   58.94886778]
Distance Ephemerides    : [29.93438305 29.93438303 29.93438273 29.93438301 29.93438287 29.934383
 29.93438307]

My Determined PHOTFLAM: [1.69927901e-18 4.72067102e-19 4.72279722e-18 2.18310084e-18
 4.74111259e-18 3.91373889e-19 4.64885482e-19]
Header PHOTFLAM       : [1.6932291e-18 4.6534907e-19 4.7187002e-18 2.1884715e-18 4.7367001e-18
 3.8794931e-19 4.5835303e-19]
Amy PHOTFLAM          : [1.639547e-18, 4.5253925e-19, 4.677e-18, 2.14434e-18, 4.653e-18, 3.8165376e-19, 4.5375198e-19]
